# Pandas 2

## Introducción


En esta notebook vamos a trabajar con los conceptos de agregación a través de `groupby` y `pivot tables`. 

Analizaremos características de precio y superficie, por barrio y por comuna, de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [1]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [2]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [3]:
data_location = "../Data/departamentos-en-venta-2016.csv"

In [4]:
get_encoding_type(data_location)

'utf-8'

In [5]:
df = pd.read_csv(data_location, sep=';', encoding='utf-8')

In [6]:
df.tail()

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
7559,NaN,NaN,42302009,VTA,DTO,37,32,U$S 84.700,1482250,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7560,NaN,NaN,42302010,VTA,DTO,44,44,U$S 99.000,1732500,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7561,NaN,NaN,42302011,VTA,DTO,45,40,U$S 95.220,1666350,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7562,NaN,NaN,42302012,VTA,DTO,44,39,U$S 95.220,1666350,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7563,"GARAY, JUAN DE AV.",612.0,42394651,VTA,DTO,51,46,U$S 147.619,2583332,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42394...,NaN,NaN,"GARAY, JUAN DE AV. 612",SAN TELMO,COMUNA 01,1153.0,C1153ABR,-34.624541,-58.374337


In [7]:
df.shape

(7564, 29)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7564 entries, 0 to 7563
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CALLE                    6182 non-null   object 
 1   NUMERO                   6182 non-null   float64
 2   ID_ZONAPRO               7564 non-null   int64  
 3   OPERACION                7564 non-null   object 
 4   TIPO                     7564 non-null   object 
 5   M2                       7564 non-null   int64  
 6   M2CUB                    7564 non-null   int64  
 7   PRECIOTEXT               7562 non-null   object 
 8   PRECIOARS                7564 non-null   int64  
 9   PRECIOARSM               7564 non-null   int64  
 10  DOLARES                  7564 non-null   int64  
 11  U_S_M2                   7564 non-null   int64  
 12  AMBIENTES                7564 non-null   int64  
 13  ANTIGUEDAD               7564 non-null   int64  
 14  BAÑOS                   

## Ejercicio 2  - Promedio

¿Cuál es el promedio valor del precio por $m^2$ en dolares?

Para responder esta pregunta vamos a:

1) Crear un obeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)

3) Crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

4) Calcular la media de los valores de PRECIOTEXTM


In [9]:
def limpiar_precio(precio):
    precio = str(precio)
    valor = precio
    # Está en dólares?
    if precio.startswith('U$S '):
        valor = precio.lstrip('U$S ')
    # Remuevo el punto separador de miles
    if '.' in precio:
        valor = valor.replace('.','')
    # Elimino los valores en pesos
    if precio.startswith('$'):
        valor = None
    return valor

In [10]:
df['PRECIOTEXT_CLEAN'] = df['PRECIOTEXT'].apply(limpiar_precio).astype('float')

In [11]:
df['PRECIOTEXTM'] = df['PRECIOTEXT_CLEAN']/df['M2']

In [12]:
print('Promedio de valor en dólares del M2: ')
df['PRECIOTEXTM'].mean().round(2)

Promedio de valor en dólares del M2: 


2522.49

## Ejercicio 3  - Promedio por cuartil
¿Cuál es el precio promedio del metro cuadrado en dolares para cada cuartil de superficie (campo M2) de las viviendas en CABA?

Comenzamos calculando los cuartilos de superficie.

In [13]:
q1_m2 = df['M2'].quantile(0.25)
q2_m2 = df['M2'].quantile(0.50)
q3_m2 = df['M2'].quantile(0.75)
q4_m2 = df['M2'].quantile(1)
q1_m2, q2_m2, q3_m2, q4_m2

(41.0, 54.0, 80.0, 730.0)

In [14]:
df_q1 = df[df['M2']<=q1_m2]
df_q2 = df[(df['M2']>q1_m2) & (df['M2']<=q2_m2)]
df_q3 = df[(df['M2']>q2_m2) & (df['M2']<=q3_m2)]
df_q4 = df[df['M2']>q3_m2]

In [15]:
df_q1['PRECIOTEXTM'].mean().round(2), df_q2['PRECIOTEXTM'].mean().round(2), df_q3['PRECIOTEXTM'].mean().round(2), df_q4['PRECIOTEXTM'].mean().round(2)

(2418.49, 2455.99, 2438.61, 2788.3)

In [16]:
# Forma alternativa con qcut y groupby
df_qs = pd.qcut(df['M2'],4)
df.groupby(df_qs)['PRECIOTEXTM'].mean().round(2)

M2
(14.999, 41.0]    2418.49
(41.0, 54.0]      2455.99
(54.0, 80.0]      2438.61
(80.0, 730.0]     2788.30
Name: PRECIOTEXTM, dtype: float64

## Ejercicio 3  - Promedio por barrio

¿Cuál es la media de precio por metro cuadrado en dolares para cada barrio de la CABA? 

Ordenemos los datos para indicar cuál es el barrio más caro.

Resolvamos el cálculo tanto con groupby como con pivot tables

In [72]:
barrio_group = df.groupby('BARRIO')
barrio_group['PRECIOTEXTM']\
            .mean()\
            .sort_values(ascending=False)

BARRIO
PUERTO MADERO        5412.468738
PALERMO              3136.014984
BELGRANO             3110.065913
RETIRO               2941.052687
RECOLETA             2876.201544
PARQUE CHACABUCO     2645.412539
COLEGIALES           2636.723717
NUÑEZ                2597.472491
COGHLAN              2451.120924
CABALLITO            2412.777566
VILLA ORTUZAR        2408.765643
VILLA URQUIZA        2402.505658
SAAVEDRA             2365.061414
MONSERRAT            2349.127916
CHACARITA            2338.939470
VILLA CRESPO         2331.951318
AGRONOMIA            2276.703779
ALMAGRO              2252.269793
VILLA DEL PARQUE     2240.717867
BOEDO                2215.391462
CONSTITUCION         2197.241584
VILLA DEVOTO         2190.415302
SAN NICOLAS          2186.049883
BARRACAS             2166.146707
VILLA PUEYRREDON     2159.102203
SAN TELMO            2145.817227
PARQUE CHAS          2137.475822
BALVANERA            1988.284935
BOCA                 1963.575157
VILLA LURO           1960.143435
MON

In [18]:
pd.pivot_table(df, values='PRECIOTEXTM', index='BARRIO')\
                .sort_values(by='PRECIOTEXTM', ascending=False)

,PRECIOTEXTM
BARRIO,
PUERTO MADERO,5412.468738
PALERMO,3136.014984
BELGRANO,3110.065913
RETIRO,2941.052687
RECOLETA,2876.201544
PARQUE CHACABUCO,2645.412539
COLEGIALES,2636.723717
NUÑEZ,2597.472491
COGHLAN,2451.120924


## Ejercicio 4  - Dispersión de precios por barrio

**4.a** ¿En qué barrio hay una mayor dispersión en el valor del metro cuadrado en dolares? Ordenar los valores para identificar el mayor.

Ayuda: Calculamos dispersión como el desvío estandar de un grupo dividido la media de ese grupo grupo

Vamos a calcular esto de dos maneras distintas:

1) Calculamos la instancia de Series que tiene la media por grupo. Calculamos la instancia de Series que tiene el desvío estandar por grupo. Las dividimos. Ordenamos

2) Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos.

**4.b** Cuál les parece que será más eficiente? Probemoslo con %timeit, para eso definamos dos funciones que encapsulen el código de 1) y 2).

**4.c** ¿Por qué Paternal está devolviendo NaN? ¿Y por qué Villa Soldati devuelve 0?

Nota: tengan en cuenta que `mean` y `std` sobre una instancia de `DataFrameGroupBy` excluye los valores nulos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html

In [73]:
def dispersion_1(df):
    grouped_barrio = df['PRECIOTEXTM'].agg(['mean', 'std'])
    grouped_barrio['dispersion']= grouped_barrio['std']/grouped_barrio['mean']
    grouped_barrio = grouped_barrio.sort_values(by='dispersion', ascending=False)
    return grouped_barrio

def dispersion_2(df):
    grouped_barrio = df['PRECIOTEXTM'].apply(lambda x: x.std()/x.mean())
    grouped_barrio = grouped_barrio.sort_values(ascending=False)
    return grouped_barrio

In [74]:
comp_dispersion = pd.DataFrame([dispersion_1(barrio_group)['dispersion'], dispersion_2(barrio_group)], )\
                                .transpose()\
                                .rename(columns={'dispersion':'dispersion_con_agg', 'PRECIOTEXTM':'dispersion_con_apply'})
comp_dispersion

,dispersion_con_agg,dispersion_con_apply
BARRIO,,
VILLA DEVOTO,0.440995,0.440995
BOCA,0.425274,0.425274
MONSERRAT,0.321376,0.321376
PALERMO,0.308486,0.308486
BALVANERA,0.300993,0.300993
NUÑEZ,0.295307,0.295307
VELEZ SARSFIELD,0.275630,0.275630
RECOLETA,0.263813,0.263813
CHACARITA,0.249468,0.249468


In [75]:
%%timeit
dispersion_1(barrio_group)

2.24 ms ± 61.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
%%timeit
dispersion_2(barrio_group)

14.5 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vamos a ver cuánto da si usamos la lambda directamente, sin pasar por dispersion_2:

In [77]:
%%timeit
grouped_barrio = barrio_group['PRECIOTEXTM']\
                        .apply(lambda x: x.std()/x.mean())\
                        .sort_values(ascending=False)

14.4 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Conclusión:** Es mucho más rápido con el groupby y funciones de agregación de pandas que un apply. Y parece que si se hace una lambda directa en vez de a través de una función es más rápido, pero no se diferencian significativamente acá (habría que probar con más registros)

Veamos por qué Paternal devuelve NaN y Villa Soldati que devuelve 0.


In [68]:
df[df['BARRIO'] == 'PATERNAL']

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD,PRECIOTEXT_CLEAN,PRECIOTEXTM
3976,"OROÑO, NICASIO",2100.0,41918546,VTA,DTO,77,69,U$S 132.500,2318750,0,...,NaN,"OROÑO, NICASIO 2100",PATERNAL,COMUNA 15,1416.0,C1416BZT,-34.602445,-58.461133,132500.0,1720.779221


In [65]:
df[df['BARRIO'] == 'VILLA SOLDATI']

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD,PRECIOTEXT_CLEAN,PRECIOTEXTM
3059,"ACOSTA, MARIANO",3500.0,41892130,VTA,DTO,80,80,U$S 65.000,1137500,0,...,NaN,"ACOSTA, MARIANO 3500",VILLA SOLDATI,COMUNA 08,1437.0,C1437CEB,-34.66493,-58.44317,65000.0,812.5
3060,"ACOSTA, MARIANO",3500.0,41892131,VTA,DTO,80,80,U$S 65.000,1137500,0,...,NaN,"ACOSTA, MARIANO 3500",VILLA SOLDATI,COMUNA 08,1437.0,C1437CEB,-34.66493,-58.44317,65000.0,812.5


**Conclusión:** Paternal al ser sólo un registro no tiene dessvío estándar y por lo tanto ese dato es NaN. En cambio Vilal Soldati sus dos registros son un duplicado, por lo que su STD es 0 (en este caso todo el registro, pero podría serlo sólo en el precio y seguiría ocurriendo lo mismo). 

## Ejercicio 5  - Superficie por barrio

Calculemos la mediana de superficie por barrio para determinar en qué barrio los departamentos son más grandes.

Hagamos el cálculo con pivot_table y groupby


In [104]:
# con pivot table
m2_barrio_pivot = df.pivot_table(values='M2', index='BARRIO', aggfunc='median')\
                                .sort_values(by='M2', ascending=False)\
                                .rename(columns={'M2':'con_pivot'})

# Con groupby
m2_barrio_groupby = barrio_group[['M2']]\
                                .median()\
                                .sort_values(by='M2', ascending=False)\
                                .rename(columns={'M2':'con_groupby'})

In [114]:
iguales

BARRIO
PUERTO MADERO        True
VILLA SOLDATI        True
PATERNAL             True
MATADEROS            True
COGHLAN              True
BELGRANO             True
RECOLETA             True
VELEZ SARSFIELD      True
FLORES               True
VILLA DEL PARQUE     True
VERSALLES            True
COLEGIALES           True
RETIRO               True
BARRACAS             True
VILLA LURO           True
PARQUE CHAS          True
VILLA LUGANO         True
NUÑEZ                True
BOCA                 True
LINIERS              True
PALERMO              True
FLORESTA             True
PARQUE PATRICIOS     True
VILLA DEVOTO         True
BALVANERA            True
SAAVEDRA             True
CABALLITO            True
SAN TELMO            True
ALMAGRO              True
VILLA ORTUZAR        True
VILLA URQUIZA        True
SAN CRISTOBAL        True
CONSTITUCION         True
VILLA PUEYRREDON     True
MONTE CASTRO         True
VILLA REAL           True
BOEDO                True
SAN NICOLAS          True
VILLA

In [117]:
iguales = m2_barrio_groupby['con_groupby']==m2_barrio_pivot['con_pivot']
pd.concat([m2_barrio_groupby, m2_barrio_pivot, iguales], axis=1)\
            .rename(columns={0:'iguales?'})

,con_groupby,con_pivot,iguales?
BARRIO,,,
PUERTO MADERO,117.0,117.0,True
VILLA SOLDATI,80.0,80.0,True
PATERNAL,77.0,77.0,True
MATADEROS,75.0,75.0,True
COGHLAN,74.5,74.5,True
BELGRANO,74.0,74.0,True
RECOLETA,74.0,74.0,True
VELEZ SARSFIELD,73.0,73.0,True
FLORES,71.0,71.0,True


## Ejercicio 6  - Tabla resumen

Generemos un `DataFrame` que agregue la información del precio por M2 en dolares (`PRECIOTEXTM`), ambientes (`AMBIENTES`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

Ayuda: usar pivot_table

In [123]:
df.pivot_table(index=['COMUNA', 'BARRIO'], aggfunc={'PRECIOTEXTM':['mean', 'std'], 'AMBIENTES':['mean', 'std']})

AMBIENTES            PRECIOTEXTM             
                                 mean       std         mean          std
COMUNA    BARRIO                                                         
COMUNA 01 CONSTITUCION       2.208696  0.893403  2197.241584   457.810983
          MONSERRAT          2.100000  1.087115  2349.127916   754.954231
          PUERTO MADERO      2.678571  0.765320  5412.468738  1268.483199
          RETIRO             2.892473  1.684084  2941.052687   718.493943
          SAN NICOLAS        2.190698  1.232696  2186.049883   372.454324
          SAN TELMO          2.256410  0.594623  2145.817227   533.737353
COMUNA 02 RECOLETA           2.931204  1.353770  2876.201544   758.780248
COMUNA 03 BALVANERA          2.476636  1.046755  1988.284935   598.460600
          SAN CRISTOBAL      2.397869  0.621211  1914.203956   337.113408
COMUNA 04 BARRACAS           2.583333  1.088918  2166.146707   459.975468
          BOCA               2.312500  1.138347  1963.575157   835.057816
          NUEVA POMPEYA      2.000000  0.000000  1622.727273   109.280139
          PARQUE PATRICIOS   2.208955  0.640485  1943.392373   223.179437
COMUNA 05 ALMAGRO            2.370149  1.094490  2252.269793   431.445048
          BOEDO              2.032258  1.224306  2215.391462   333.432291
COMUNA 06 CABALLITO          2.528771  0.947383  2412.777566   403.811538
COMUNA 07 FLORES             2.806452  0.875488  1905.204204   472.870879
          PARQUE CHACABUCO   2.330579  0.541969  2645.412539   329.268919
COMUNA 08 VILLA LUGANO       2.333333  1.211060  1138.726815   153.217663
          VILLA SOLDATI      4.000000  0.000000   812.500000     0.000000
COMUNA 09 LINIERS            2.290323  0.662511  1885.041244   374.817469
          MATADEROS          2.727273  0.674200  1804.158874   414.312201
          PARQUE AVELLANEDA  2.000000  0.000000  1806.324289   108.443814
COMUNA 10 FLORESTA           2.277778  1.017815  1711.078346   210.982377
          MONTE CASTRO       2.400000  0.709721  1952.634631   368.079223
          VELEZ SARSFIELD    3.200000  2.049390  1748.636059   481.976635
          VERSALLES          3.000000  0.000000  1926.168413   166.805407
          VILLA LURO         2.705882  1.046704  1960.143435   445.368473
          VILLA REAL         2.666667  0.577350  1738.200049    62.479211
COMUNA 11 VILLA DEL PARQUE   2.692308  1.319674  2240.717867   403.395202
          VILLA DEVOTO       2.653846  1.809802  2190.415302   965.962914
          VILLA GRAL. MITRE  2.285714  0.673024  1928.578367   325.546393
          VILLA SANTA RITA   2.137931  0.789400  1907.998258   201.952963
COMUNA 12 COGHLAN            2.684211  0.808912  2451.120924   402.036953
          SAAVEDRA           2.204819  1.000110  2365.061414   501.751822
          VILLA PUEYRREDON   2.000000  0.946864  2159.102203   322.435274
          VILLA URQUIZA      2.157609  1.009326  2402.505658   444.782207
COMUNA 13 BELGRANO           2.767442  1.125982  3110.065913   762.970942
          COLEGIALES         2.726316  1.046190  2636.723717   573.635491
          NUÑEZ              2.595745  0.999848  2597.472491   767.050772
COMUNA 14 PALERMO            2.491256  1.247419  3136.014984   967.417254
COMUNA 15 AGRONOMIA          1.833333  0.983192  2276.703779   192.974291
          CHACARITA          1.764706  1.087901  2338.939470   583.490399
          PARQUE CHAS        1.937500  0.573730  2137.475822   290.456966
          PATERNAL           3.000000       NaN  1720.779221          NaN
          VILLA CRESPO       2.255639  0.838384  2331.951318   425.238932
          VILLA ORTUZAR      2.264151  0.593262  2408.765643   282.605371